In [2]:
import numpy as np
import cv2

In [11]:
cap=cv2.VideoCapture(0) #camera feed
target=cv2.imread('21.jpeg') #image target
target=cv2.resize(target,(726,512)) #reshaping
vid=cv2.VideoCapture('2.mp4') # video

In [ ]:
cv2.imshow('target',target)
cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
ht,wt,ct=target.shape #getting target dimensions

In [4]:
success,frame=vid.read() #getting frame of video
imgvid=cv2.resize(frame,(ht,wt)) #resizing frame to image dimension

In [5]:
cv2.imshow('image video',imgvid)
cv2.waitKey()
cv2.destroyAllWindows()

**ORB Detection**

In [5]:
orb=cv2.ORB_create(nfeatures=1000) #feature detector to detect atmost 1000

In [5]:
#Key points and the Descriptors for the Target Image.
key,des=orb.detectAndCompute(target,None)

In [8]:
cv2.imshow('detected features',cv2.drawKeypoints(target,kps,None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))
cv2.waitKey()
cv2.destroyAllWindows()

**SURF Detection**

In [6]:
surf=cv2.xfeatures2d.SURF_create()
key,des=surf.detectAndCompute(target,None)

In [7]:
cv2.imshow('surf keypoints',cv2.drawKeypoints(target,key,None))
cv2.waitKey()
cv2.destroyAllWindows()

**SIFT Detection**

In [7]:
sift=cv2.xfeatures2d.SIFT_create()
key,des=sift.detectAndCompute(target,None)

In [28]:
cv2.imshow('sift keypoints',cv2.drawKeypoints(target,key,None))
cv2.waitKey()
cv2.destroyAllWindows()

### Feature Matching

In [12]:
algo="sift"
method={"orb":orb,"surf":surf,"sift":sift}
kp1,des=method[algo].detectAndCompute(target,None)
while True:
#     matrix=[]
    success,frame=cap.read()
    imgaug=frame.copy()
    r,f=vid.read()
    if not r:
        vid=cv2.VideoCapture('2.mp4')
        r,f=vid.read()
    kp2,des2=method[algo].detectAndCompute(frame,None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des, des2, k=2)
    good = []
    for m, n in matches:
        if m.distance < 0.80 * n.distance:
            good.append(m)
#     print(len(good))
    thresholds={"orb":12,"surf":200,"sift":140}
    #homography : to find relevant relations btw src and target points
    if len(good)>=thresholds[algo]:
        srcPts=np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        desPts=np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
        matrix,mask=cv2.findHomography(srcPts,desPts,cv2.RANSAC,5.0)
#     print(matrix)
        pts=np.float32([[0,0],[0,ht],[wt,ht],[wt,0]]).reshape(-1,1,2)
        dst=cv2.perspectiveTransform(pts,matrix) #perspective transform to get the bounding box
        bbox=cv2.polylines(frame,[np.int32(dst)],True,(0,255,0),3)
        f=cv2.resize(f,(wt,ht))
        vid_warp=cv2.warpPerspective(f,matrix,(bbox.shape[1],bbox.shape[0]))
        mask_new=np.zeros((bbox.shape[0],bbox.shape[1]),np.uint8)
        cv2.fillPoly(mask_new,[np.int32(dst)],(255,255,255))
        
    else:
        bbox=frame
        vid_warp=frame
        mask_new=np.zeros((bbox.shape[0],bbox.shape[1]),np.uint8)
        
    features=cv2.drawMatches(target,kp1,frame,kp2,good,None,flags=2)

    maskInv=cv2.bitwise_not(mask_new)
    imgaug=cv2.bitwise_and(imgaug,imgaug,mask=maskInv)
    imgaug=cv2.bitwise_or(imgaug,vid_warp)
#     cv2.imshow('feature match',features)
    cv2.imshow('vision ar',imgaug)
#     cv2.imshow('mask',maskInv)
#     cv2.imshow('bounding box',bbox)
#     cv2.imshow('warp',vid_warp)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### Descriptor matching (multiple images)

In [3]:
#Descriptor matching
import cv2
import numpy as np
from time import perf_counter

sift=cv2.xfeatures2d.SIFT_create()    #creating sift feature matcher
kp,desc=[],[]                         #empty list to contain all the key points and descriptor
for i in range(9):                    #looping through range of images
    img=cv2.imread(f'{i+1}.jpeg')     #reading image
    img=cv2.resize(img,(726,512))     #resizing image
    key,des=sift.detectAndCompute(img,None) #getting kp's and des
    kp.append(key)                    #appending keys and descriptors to the lists
    desc.append(des)  
ht,wt,ct=img.shape                    #getting image shape 
cap=cv2.VideoCapture(0)               #create camera feed object
thresh=145                            #setting a threshold for matches
i=0                                   #setting initial index to zero
match=False                           #feed loop yet to start so match = false

while True:                           #camera frame feed loop
#     matrix=[]
    if match:                         #if match is already true then continue reading the target video frames
        r,f=vid.read()
    success,frame=cap.read()          #getting camera frame
    t1_start = perf_counter()         #time counter start
    imgaug=frame.copy()               #copy of the frame
    kp2,des2=sift.detectAndCompute(frame,None) #getting kp and des for the frame
    bf = cv2.BFMatcher()              #create brute force matcher object
    if i>=9:                          #if index exceeds 9 then switch back to 0
        i=0
    matches = bf.knnMatch(desc[i], des2, k=2) #getting matches by comparing descriptors with knn 
    good = []                         #container list to store matches
    for m, n in matches:              #for match from img if the distance is less than 80% of distance from feed then append that match
        if m.distance < 0.80 * n.distance:
            good.append(m)
    if len(good)>=thresh:             #if good matches are greater than threshold
        t2_end = perf_counter()       #its a match check point the timer again
        print(t2_end-t1_start)        
#         print(len(good))
#         cap.release()
#         break
        if not match:                 #if match was false 
            vid=cv2.VideoCapture(f'{i+1}.mp4') #read from video 
            img=cv2.imread(f'{i+1}.jpeg')
            r,f=vid.read()
        match=True                    #set match to true
        srcPts=np.float32([kp[i][m.queryIdx].pt for m in good]).reshape(-1,1,2)  #get the source points i.e image points
        desPts=np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)    #get the destination points i.e points from video
        matrix,mask=cv2.findHomography(srcPts,desPts,cv2.RANSAC,5.0)             #find homography matrix
#     print(matrix)
        pts=np.float32([[0,0],[0,ht],[wt,ht],[wt,0]]).reshape(-1,1,2)            #create points for  perspective transform
        dst=cv2.perspectiveTransform(pts,matrix) #perspective transform to get the bounding box
        bbox=cv2.polylines(frame,[np.int32(dst)],True,(0,255,0),3)               #creating a bounding box with poly lines inside frame
        f=cv2.resize(f,(wt,ht))       #resize the video frame to the size of image
        vid_warp=cv2.warpPerspective(f,matrix,(bbox.shape[1],bbox.shape[0]))     #perspective warp the video frame for localization
        mask_new=np.zeros((bbox.shape[0],bbox.shape[1]),np.uint8)                #create a mask of shape of bounding box
        cv2.fillPoly(mask_new,[np.int32(dst)],(255,255,255))                     #fill the remaining mask with white
    else:
        match=False                                                              #if matches are less than thresh value
        bbox=frame                                                               #set match to False
        vid_warp=frame
        mask_new=np.zeros((bbox.shape[0],bbox.shape[1]),np.uint8)               
#     features=cv2.drawMatches(img,kp[i],frame,kp2,good,None,flags=2)              #optional to drow matches
    maskInv=cv2.bitwise_not(mask_new)                                            #inverse the color of created mask by bitwise not
    imgaug=cv2.bitwise_and(imgaug,imgaug,mask=maskInv)                           #bitwise &, the frame and the masked box
    imgaug=cv2.bitwise_or(imgaug,vid_warp)                                       #bitwise or ,the masked video and the frame 
    cv2.imshow('vision ar',imgaug)                                               #  show frame
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    if not match:                                                                #if not a match the increment the index
        i+=1
# img=cv2.imread(f'{i+1}.jpeg')
# print("Avg. time", t2_end - t1_start)
# cv2.imshow('matched',cv2.resize(img,(726,512)))
# cv2.waitKey()
cap.release()
cv2.destroyAllWindows()

0.12529229199935799
0.12963616299748537
0.14443002500047442
0.13430013300239807
0.13514467799905105
0.13989165300154127
0.13741683800253668
0.14482785200016224
0.14502820300185704
0.13879069500035257
0.14066317600008915
0.14135330999852158
0.13494770000033895
0.1410260209995613
0.1391724040004192
0.13585393099856446
0.13857981800174457
0.13047427399942535
0.13568982200013124
0.1373684210011561
0.1392536100029247
0.14167888100200798
0.13821302699943772
0.13984913600143045
0.1237938039994333
0.12121020400081761


In [47]:
des.shape

(2277, 128)

In [49]:
matrix.shape

(3, 3)

In [50]:
mask.shape

(378, 1)

### Barcode/QRcode generation and detection

In [2]:
import barcode 
from barcode.writer import ImageWriter
from pyzbar import pyzbar

In [ ]:
bar=barcode.get('code128','vaibhav haswani123',writer=ImageWriter()) #generating custom barcode 
bar.get_fullcode()
bar.save('bar1')

In [11]:
import pyqrcode
from PIL import Image
url = pyqrcode.QRCode('http://www.eqxiu.com',error = 'H')
url.png('test.png',scale=10)
im = Image.open('test.png')
im = im.convert("RGBA")
logo = Image.open('1.jpeg')
box = (135,135,235,235)
im.crop(box)
region = logo
region = region.resize((box[2] - box[0], box[3] - box[1]))
im.paste(region,box)
im.show()

In [15]:
import pyqrcode
from PIL import Image

# Generate the qr code and save as png
qrobj = pyqrcode.create('https://stackoverflow.com')
with open('test.png', 'wb') as f:
    qrobj.png(f, scale=10)

# Now open that png image to put the logo
img = Image.open('test.png')
width, height = img.size

# How big the logo we want to put in the qr code png
logo_size = 2000

# Open the logo image
logo = Image.open('logo.jpeg')

# Calculate xmin, ymin, xmax, ymax to put the logo
xmin = ymin = int((width / 2) - (logo_size / 2))
xmax = ymax = int((width / 2) + (logo_size / 2))

# resize the logo as calculated
logo = logo.resize((xmax - xmin, ymax - ymin))

# put the logo in the qr code
img.paste(logo, (xmin, ymin, xmax, ymax))

In [34]:

import qrcode
from PIL import Image

Logo_link = '1.jpeg'#input('Please enter the path for your logo: ')

logo = Image.open(Logo_link)
basewidth = 250 #int(input('Please Enter the Base Width [Default=75]: ') or '75')
wpercent = (basewidth/float(logo.size[0]))
hsize = int((float(logo.size[1])*float(wpercent)))
logo = logo.resize((basewidth,hsize), Image.ANTIALIAS)
qr_big = qrcode.QRCode(
    error_correction=qrcode.constants.ERROR_CORRECT_H,
    box_size=20,
    border=1
)
data='hello world' #data for qr#nput('Please enter the URL after clicking: ')
qr_big.add_data(data)
qr_big.make()
qr_color = 'blue' #input('What Color Would You like Your QR Code to be? [Default=Black]: ' or 'Black')
img_qr_big = qr_big.make_image(fill_color=qr_color, back_color="white").convert('RGB')
pos = ((img_qr_big.size[0] - logo.size[0]) // 2, (img_qr_big.size[1] - logo.size[1]) // 2)
img_qr_big.paste(logo, pos)
save_path = 'logo.png' #input('Please enter the filename you want for your branded qr code: ')
img_qr_big.save(save_path)

In [36]:
img=cv2.imread('logo.png')

In [39]:
i=cv2.resize(img,(512,256))

In [ ]:
cv2.imshow('reshaped',i)
cv2.waitKey()
cv2.destroyAllWindows()

### Aruco Markers

In [1]:
import cv2
import cv2.aruco as aruco